In [85]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import os
from random import randint
import time
from geopy.geocoders import Nominatim
from pygeocoder import Geocoder
from tzwhere import tzwhere

geolocator = Nominatim()
chromedriver = 'C:/Users/vales/Anaconda3/Lib/ChromeDriver/chromedriver.exe'
os.environ['webdriver.chrome.driver'] = chromedriver

In [73]:
tournament = 'Pebble Beach Pro AM' + ' PGA'

def find_tournament_page(tournament):
    url = 'https://www.wikipedia.com'
    browser = webdriver.Chrome()
    browser.get(url)

    search_box = browser.find_element_by_xpath('//*[@id="searchInput"]')
    search_box.send_keys(tournament)

    search_btn = browser.find_element_by_xpath('//button[@class="pure-button pure-button-primary-progressive"]')
    search_btn.click()

    time.sleep(2)

    for i in range(5):
        search_results = browser.find_elements_by_xpath('//div[@class="mw-search-result-heading"]/a')
        time.sleep(1)
        search_results[i].click()    
        try:
            course_heading = browser.find_element_by_xpath('//th[@scope="row" and contains(text(), "Course(s)")]')
            courses = course_heading.find_element_by_xpath('..').text.split('\n')
            courses[0] = courses[0].replace('Course(s) ', '')
            links = []
            for course in courses:
                try:
                    course_link = browser.find_element_by_xpath('//a[contains(text(), "' + course + '")]').get_attribute('href')
                    links.append({course: course_link})
                except Exception as e:
                    links.append({course: 'nan'})
            browser.close()
            return links
        except Exception as e:
            browser.back()
       

    print('tournament page not found')
    
def parse_links(tournament):
    links = find_tournament_page(tournament)
    browser = webdriver.Chrome()
    
    for link in links:
        course = list(link.keys())[0]
        url = link[course]
        
        course_dict = []
        if url != 'nan':
            browser.get(url)

            lat_lon_link = browser.find_element_by_xpath('//span[@class="geo-dec"]')
            lat_lon_link.click()

            time.sleep(1)

            lat = float(browser.find_element_by_xpath('//span[@class="latitude"]').text)
            lon = float(browser.find_element_by_xpath('//span[@class="longitude"]').text)

            location = Geocoder.reverse_geocode(lat, lon)
            city = location.city
            state = location.state
            country = location.country

            tzwhere = tzwhere.tzwhere()
            timezone = tzwhere.tzNameAt(lat, lon)
            
            temp_dict = {'name': course, 'latitude': lat, 'longitude': lon, 'city': city, 'state': state, 
                          'country': country, 'timezone': timezone}
            
            course_dict.append(temp_dict) 
            
    return course_dict

In [74]:
parse_links(tournament)

36.56 -121.95
36.582 -121.957
36.601431 -121.948457


In [80]:
lat = 36.601431
lon = -121.948457

location = Geocoder.reverse_geocode(lat, lon)



In [86]:
tzwhere = tzwhere.tzwhere()
timezone = tzwhere.tzNameAt(lat, lon)

timezone

'America/Los_Angeles'